In [ ]:
%config Completer.use_jedi = False

In [ ]:
import sys
from fastai.vision.all import *
sys.path.append("/kaggle/input/pointnet/models")
sys.path.append("/kaggle/input/helmet-assignment-helpers/helmet-assignment-main")
from helmet_assignment.features import add_track_features
from helmet_assignment.score import NFLAssignmentScorer
from tqdm.notebook import tqdm

from pointnet_utils import PointNetEncoder, feature_transform_reguliarzer

In [ ]:
class cameraRotationNet(nn.Module):
    def __init__(self, in_ch = 3, n_cls = 2):
        super().__init__()
        self.encoder1 = PointNetEncoder(global_feat=True, feature_transform=True, channel=in_ch)
        self.encoder2 = PointNetEncoder(global_feat=True, feature_transform=True, channel=in_ch)
        head = create_head(1024*2, n_cls, concat_pool=False)
        head[0] = Identity()
        head[1] = Identity()
        self.head = head
    
    def forward(self, x1, x2):
        x1, _, tf1 = self.encoder1(x1)
        x2, _, tf2 = self.encoder2(x2)
        x = torch.cat([x1, x2], 1)
        x = self.head(x)
        return x, tf1, tf2

In [ ]:
class myLoss(Module):
    def __init__(self, mat_diff_loss_scale=0.001):
        super().__init__()
        self.mat_diff_loss_scale = mat_diff_loss_scale

    def forward(self, pred, target):
        loss = F.cross_entropy(pred[0], target)
        mat_diff_loss1 = feature_transform_reguliarzer(pred[1])
        mat_diff_loss2 = feature_transform_reguliarzer(pred[2])

        total_loss = loss + (mat_diff_loss1 + mat_diff_loss2) * self.mat_diff_loss_scale
        return total_loss

In [ ]:
class track_data():
    def __init__(self, is_sub, normalize = False, flip_y = True):
        
        if is_sub:
            data = pd.read_csv('/kaggle/input/nfl-health-and-safety-helmet-assignment/test_player_tracking.csv')
        else:
            data = pd.read_csv('/kaggle/input/nfl-health-and-safety-helmet-assignment/train_player_tracking.csv')
        print('Adding track features... ', end="", flush=True)
        data = add_track_features(data)
        print('Done!')
        data = data.query("est_frame > 0").copy()
        
        if normalize:
            print('Normalizing x-coordinate by frame for all videos... ', end="", flush=True)
            data['x'] = data.groupby('est_frame')['x'].transform(lambda x: (x - x.mean())/x.std())
            print('Done!')
            print('Normalizing y-coordinate by frame for all videos... ', end="", flush=True)
            data['y'] = data.groupby('est_frame')['y'].transform(lambda x: (x - x.mean())/x.std())
            print('Done!')
            
        if flip_y:
            data['y'] = - data['y']
            
        self.data = data
        
    def match_video_frames(self, video, frames):
        game_play = '_'.join(video.split('_')[:-1])
        data = (self.data
            .query(f'game_play == "{game_play}"')
            .reset_index(drop=True)
            .rename({'est_frame':'frame'}, axis = 1)
            .pivot('frame', 'player', ['x','y'])
            .reindex(frames)
            .interpolate(limit_direction='both')
            .unstack()
            .unstack(level=0)
            .sort_index(axis=0, level=1)
            .swaplevel(0, 1))
        return data

## Building the Dataframes

In [ ]:
td = track_data(False)

In [ ]:
bbox_df = pd.read_csv('/kaggle/input/nfl-health-and-safety-helmet-assignment/train_labels.csv')
bbox_df['x'] = bbox_df['left'] + bbox_df['width']/2
bbox_df['y'] = bbox_df['top'] + bbox_df['height']/2
bbox_df['game_play'] = bbox_df['video_frame'].apply(lambda x: '_'.join(x.split('_')[:2]))
bbox_df = bbox_df.query('view == "Endzone"')

In [ ]:
_video_frame = []
_video_xyz = []
_gt_label = []
for video_frame, data in tqdm(bbox_df.groupby('video_frame')):
    _video_frame.append(video_frame)
    _video_xyz.append(np.array([
        (data['x'].values - data['x'].mean())/data['x'].std(),
        (data['y'].values - data['y'].mean())/data['y'].std(),
        [0] * len(data)
    ]).T)
    _gt_label.append(data['label'].values)
video_df = pd.DataFrame({
    'video_frame':_video_frame,
    'video_xyz':_video_xyz,
    'gt_label':_gt_label
})
video_df.tail(2)

In [ ]:
_track_frame = []
_track_xyz = []
_track_label = []
for video, data in tqdm(bbox_df.groupby('video')):
    frames = data['frame'].unique()
    video[:-4]
    _df = td.match_video_frames(video, frames).reset_index()
    for frame, data in _df.groupby('frame'):
        _track_frame.append(video[:-4] + '_' + str(frame))
        _track_xyz.append(np.array([
             (data['x'].values - data['x'].mean())/data['x'].std(),
            -(data['y'].values - data['y'].mean())/data['y'].std(),
             [0] * len(data)
        ]).T)
        _track_label.append(data['player'].values)
track_df = pd.DataFrame({
    'video_frame':_track_frame,
    'track_xyz':_track_xyz,
    'track_label':_track_label
})
track_df.tail(2)

In [ ]:
df = pd.merge(video_df, track_df, on = 'video_frame')
M = []
for _, data in tqdm(df.iterrows(), total=len(df)):
    a = data['gt_label']
    b = data['track_label']
    M.append(np.tile(a, (len(b), 1)).T == np.tile(b, (len(a), 1)))
df['match_matrix'] = M
a = [i for i, x in enumerate(df['video_xyz']) if np.isnan(x).any()]
df = df.drop(a).reset_index(drop = True)
# cam_rot = pd.read_csv('/kaggle/input/nfl-camera-rotation-dataset-builder/camera_rotation.csv')
cam_rot_end = pd.read_csv('/kaggle/input/nlf-helmet-safety-camera-rotations/NFL-rotations-plays.csv')
cam_rot_end['video'] = cam_rot_end['play'] + '_Endzone'
cam_rot_end['angle'] = cam_rot_end['Endzone']
cam_rot_sid = pd.read_csv('/kaggle/input/nlf-helmet-safety-camera-rotations/NFL-rotations-plays.csv')
cam_rot_sid['video'] = cam_rot_sid['play'] + '_Sideline'
cam_rot_sid['angle'] = cam_rot_sid['Sideline']
cam_rot = pd.concat([cam_rot_end, cam_rot_sid]).drop(['Sideline', 'Endzone'], axis = 1)
df['video'] = df['video_frame'].apply(lambda x: '_'.join(x.split('_')[:-1]))
df['view'] = df['video'].apply(lambda x: x.split('_')[-1])
df = pd.merge(df, cam_rot, on = 'video')
df = df.query('view == "Endzone"')
df.tail(2)

### K-fold split

In [ ]:
df['game'] = df['video_frame'].apply(lambda x: x.split('_')[0])
games = df['game'].unique()
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=6, shuffle=True)
folds = kf.split(games, games)
folds = [games[f] for _,f in folds]

df['fold'] = [((np.vstack(folds) == g).sum(axis = 1) * np.array(range(5))).sum() for g in df['game']]
df['fold'].value_counts()

In [ ]:
def to_tensor(x: np.ndarray):
    return torch.tensor(x, dtype = torch.float)

def pad_cloud(x: torch.Tensor):
    s = x.shape
    if len(s) > 1:
        if s[0] < 22:
            x = F.pad(x, (0, 0, 0, 22 - s[0]))
        else:
            x = x[:22]
    return x
    
def pad_matrix(x: torch.Tensor):
    s = x.shape
    if s[0] < 22:
        x = F.pad(x, (0, 0, 0, 22 - s[0]))
    else:
        x = x[:22]
    if s[1] < 22:
        x = F.pad(x, (0, 22 - s[0], 0, 0))
    else:
        x = x[:,:22]
    return x

def transpose(x: torch.Tensor):
    s = x.shape
    if len(s) > 1:
        return x.transpose(1,2)
    else:
        return x
    
def shuffle(x: torch.Tensor):
    order=9999
    s = x.shape
    if len(s) > 1:
        p = torch.randperm(x1.shape[-1])
        return x[:,:,p]
    else:
        return x
    
def add_noise(x: torch.Tensor):
    s = x.shape
    if len(s) > 1:
        return x + torch.randn(x.size(), device = x.device) * 0.1
    else:
        return x

In [ ]:
def myPointBlock():
    return TransformBlock(type_tfms = to_tensor, item_tfms = pad_cloud, batch_tfms = transpose)
def myMatrixBlock():
    return TransformBlock(type_tfms = to_tensor, item_tfms = pad_matrix)

In [ ]:
fold = 0
dblock = DataBlock(
    blocks = (myPointBlock, myPointBlock, CategoryBlock),
    get_x = [ColReader('video_xyz'), ColReader('track_xyz')],
    get_y = [ColReader('angle')],
#     batch_tfms = [shuffle, add_noise],
    n_inp = 2,
    splitter = IndexSplitter(df[df['fold'] == fold].index)
#     splitter = RandomSplitter(seed = 42)
)
dls = dblock.dataloaders(df, bs = 64, num_workers = 4)

In [ ]:
x1, x2, y = dls.one_batch()
model = cameraRotationNet()
res = model(x1.cpu(), x2.cpu())
res[0].shape

In [ ]:
# class detect_nan(Callback):
#     def after_batch(self):
#         if torch.isinf(self.loss) or torch.isnan(self.loss): 
#             print(self.learn.xb[0].min(), self.learn.xb[0].max(), self.learn.xb[0].mean())
#             print(self.learn.xb[1].min(), self.learn.xb[1].max(), self.learn.xb[1].mean())
#             print(self.learn.yb[0].min(), self.learn.yb[0].max(), self.learn.xb[2].mean())
#             print(self.learn.xb)
#             print(self.learn.yb)
#             plt.imshow(self.learn.yb[0])
#             raise CancelFitException

In [ ]:
from scipy.optimize import linear_sum_assignment

def myMetric(x, y):
    x = x[0].sigmoid().detach().cpu().numpy()
    match = [linear_sum_assignment(-xx) for xx in x]
    return torch.stack([yy[m].sum()/22 for yy, m in zip(y, match)]).mean()

In [ ]:
def myAccuracy(x, y):
    return accuracy(x[0], y)

In [ ]:
model = cameraRotationNet()
learn = Learner(dls, model, loss_func=myLoss(), metrics=myAccuracy, cbs = GradientClip())

In [ ]:
# learn.lr_find()

In [ ]:
learn.fit_one_cycle(20, 1e-4)